In [1]:
from distributed import Client as DaskClient
import sqlalchemy as sa

from classify.features import *

In [2]:
DaskClient()

distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-8lpn2m9h', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-dfrs4nkp', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-d5pxs3qg', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-i8jabkyk', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-gd50_eu_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-elki4mp3', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-wor

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50678,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:50698,Total threads: 1
Dashboard: http://127.0.0.1:50699/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:50684,


In [8]:
pd.set_option('max_columns', None)

In [3]:
config = ExpungeConfig.from_yaml('classify/expunge_config.yaml')

config.years_until_next_conviction_short

numpy.timedelta64(7,'Y')

In [4]:
query = (
    sa.sql.select(expunge_model)
        # Where clause just for testing, comment out for full run
        .where(
            sa.or_(
                # expunge_model.c.person_id == 127051000000102, 
                # expunge_model.c.person_id == 224010000000817,
                # expunge_model.c.person_id == 1000000000362

                expunge_model.c.person_id == 1021000000606,
                expunge_model.c.person_id == 1070000000994
            )
        )
        .order_by(
            expunge_model.c.person_id,
            expunge_model.c.HearingDate
        )
)

In [5]:
ddf = fetch_expunge_data(custom_query=query)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520


In [6]:
ddf = clean_data(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520


In [10]:
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_short,next_conviction_disqualifier_long
person_id,,,,,,,,,,,,,,,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169,Conviction,Misdemeanor,covered elsewhere,True,NaT,NaT,2011-09-08,NaT,NaT,835 days,False,False,True,True
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520,Dismissed,Misdemeanor,covered elsewhere,True,2009-05-26,NaT,2011-09-08,58 days,NaT,777 days,True,False,True,True
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered in 19.2-392.6 - B,True,2009-07-23,NaT,2014-08-28,777 days,NaT,1085 days,True,False,True,True
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered elsewhere,True,2011-09-08,NaT,2018-09-18,1085 days,NaT,1482 days,True,False,True,True
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520,Conviction,Felony,covered elsewhere,True,2014-08-28,NaT,NaT,1482 days,NaT,NaT,False,False,False,False
